In [1]:
IMAGE_SIZE = 224
NUM_CHANNELS = 3
LMDB_DIR_PATH = "/mnt/data_ssd/lmdb"
IMAGE_BASE_PATH = "/mnt/data_ssd/datasets/flickr30k"
MODEL_NAME = "efficientnet_b3"

# Compute Image Embeddings and Persist them via LMDB

In [2]:
%load_ext google.cloud.bigquery
%load_ext lab_black
%load_ext line_profiler

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os

from seefood.features import EfficientNetFeatureExtractor
from seefood.data import LMDBEmbeddingWriter
from seefood.data import ImageDataset
from seefood.data import get_default_transform

import torch
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

Using cache found in /home/mike/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
feature_extractor = EfficientNetFeatureExtractor(MODEL_NAME).to(device)

Using cache found in /home/mike/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /home/mike/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


In [7]:
DATA_FILE = f"{LMDB_DIR_PATH}/flickr_data_{MODEL_NAME}"

## Load and Shuffle Data

In [8]:
%%bigquery df --project zenscr-seefood-dev

SELECT id, image_path as image_path_suffix
FROM `zenscr-seefood-dev.flickr.flickr30k_image_path`

In [9]:
df["image_path"] = df["image_path_suffix"].map(
    lambda suffix: os.path.join(IMAGE_BASE_PATH, suffix)
)

In [10]:
df

,id,image_path_suffix,image_path
0,0,7589467042.jpg,/mnt/data_ssd/datasets/flickr30k/7589467042.jpg
1,1,5091237679.jpg,/mnt/data_ssd/datasets/flickr30k/5091237679.jpg
2,2,3472331684.jpg,/mnt/data_ssd/datasets/flickr30k/3472331684.jpg
3,3,2444527362.jpg,/mnt/data_ssd/datasets/flickr30k/2444527362.jpg
4,4,2227179257.jpg,/mnt/data_ssd/datasets/flickr30k/2227179257.jpg
...,...,...,...
31778,31778,2454054454.jpg,/mnt/data_ssd/datasets/flickr30k/2454054454.jpg
31779,31779,3017203816.jpg,/mnt/data_ssd/datasets/flickr30k/3017203816.jpg
31780,31780,297725741.jpg,/mnt/data_ssd/datasets/flickr30k/297725741.jpg
31781,31781,2009636597.jpg,/mnt/data_ssd/datasets/flickr30k/2009636597.jpg


In [19]:
df_input = df.assign(target=0.0)  # ignore target for now

In [20]:
df_input

,id,image_path_suffix,image_path,target
0,0,7589467042.jpg,/mnt/data_ssd/datasets/flickr30k/7589467042.jpg,0.0
1,1,5091237679.jpg,/mnt/data_ssd/datasets/flickr30k/5091237679.jpg,0.0
2,2,3472331684.jpg,/mnt/data_ssd/datasets/flickr30k/3472331684.jpg,0.0
3,3,2444527362.jpg,/mnt/data_ssd/datasets/flickr30k/2444527362.jpg,0.0
4,4,2227179257.jpg,/mnt/data_ssd/datasets/flickr30k/2227179257.jpg,0.0
...,...,...,...,...
31778,31778,2454054454.jpg,/mnt/data_ssd/datasets/flickr30k/2454054454.jpg,0.0
31779,31779,3017203816.jpg,/mnt/data_ssd/datasets/flickr30k/3017203816.jpg,0.0
31780,31780,297725741.jpg,/mnt/data_ssd/datasets/flickr30k/297725741.jpg,0.0
31781,31781,2009636597.jpg,/mnt/data_ssd/datasets/flickr30k/2009636597.jpg,0.0


## Write Embeddings to LMDB

In [21]:
transform = get_default_transform(IMAGE_SIZE)

dataloader = torch.utils.data.DataLoader(
    ImageDataset(df_input, transform), batch_size=64, shuffle=False, num_workers=4
)

embedding_writer = LMDBEmbeddingWriter(feature_extractor, device)
map_size = 1073741824 * 10  # 10GB

In [22]:
embedding_writer.write(DATA_FILE, dataloader, map_size)